# 分布式爬虫<br>
让爬虫更快更有效率的方法,就是采用分布式.爬虫要花大量的时间在下载网页上.利用分布式可以实现在下载网页的**同时解析**另一部分网页.<br>
下面是一个爬取[莫烦python](https://morvanzhou.github.io/)主页网站的程序(!网站的下载速度受网速影响),没有特定的任务,第一次使用普通方法爬取

In [ ]:
#准备工作以及两个任务
import multiprocessing as mp
import time
from urllib.request import urlopen,urljoin
from bs4 import BeautifulSoup
import re

base_url='https://morvanzhou.github.io/'
#因为服务器的限制,发出的请求必须是很有限的
restricted_crawl=True

#定义爬取网页任务
def crawl(url):
    back=urlopen(url)
    return back.read().decode()
#定义网页查找任务
def parse(html):
    soup=BeautifulSoup(html,'lxml')
    urls = soup.find_all('a',href=re.compile('^/.+?/$'))
    title=soup.find('h1').get_text().strip()
    #set()可以实现对括号内容去重复功能,输出正向排序
    page_urls=set(urljoin(base_url,u['href']) for u in urls)
    url=soup.find('meta',property='og:url')['content']
    return title,page_urls,url

#一次常规爬取,没有多核运算
#设置已爬取的和未爬取的,避免重复
unseen=set([base_url,])
seen=set()

#网站个数和计时
count=0
start=time.time()
#仍然有网站时爬取
while len(unseen)!=0:
    #爬取20个网站则终止
    if restricted_crawl and len(seen)>20:
        break
    #htmls存放爬取内容
    htmls=[crawl(url) for url in unseen]
    #results存放每个html对应的解析内容
    results=[parse(h) for h in htmls]
    #将unseen中的网址添加到seen中去
    seen.update(unseen)
    #unseen清空
    unseen.clear()
    for title,page_urls,url in results:
        count+=1
        print(count,title,url)
        unseen.update(page_urls-seen)
print('total time:%.1f s' %(time.time()-start))

**好像把两段程序放在一个cell里才可以正确运行,因为某些变量not callalbe**<br>
现在让我们来对比一下分布式爬取的效率

In [ ]:
#准备工作以及两个任务
import multiprocessing as mp
import time
from urllib.request import urlopen,urljoin
from bs4 import BeautifulSoup
import re

base_url='https://morvanzhou.github.io/'
#因为服务器的限制,发出的请求必须是很有限的
restricted_crawl=True

#定义爬取网页任务
def crawl(url):
    back=urlopen(url)
    return back.read().decode()
#定义网页查找任务
def parse(html):
    soup=BeautifulSoup(html,'lxml')
    urls = soup.find_all('a',href=re.compile('^/.+?/$'))
    title=soup.find('h1').get_text().strip()
    #set()可以实现对括号内容去重复功能,输出正向排序
    page_urls=set(urljoin(base_url,u['href']) for u in urls)
    url=soup.find('meta',property='og:url')['content']
    return title,page_urls,url
#设置已爬取的和未爬取的,避免重复
unseen=set([base_url,])
seen=set()

#用分布式 pool 操作,4核
pool=mp.Pool(4)
count,start=0,time.time()
while len(unseen)!=0:
    if restricted_crawl and len(seen)>20:
        break
    htmls=[pool.apply_async(crawl,args=(u,)).get() for u in unseen]
    results=[pool.apply_async(prase,args=(h,)).get() for h in htmls]
    seen.update(unseen)
    unseen.clear()
    for title,page_urls,url in results:
        count+=1
        print(count,tltle,url)
        unseen.update(page_urls-seen)
print('total time is %.2f s'%(time.time()-start))   

# 异步加载asyncio<br>
异步加载能够利用单线程实现多线程操作模式,示意图如下:
<img src="https://morvanzhou.github.io/static/results/scraping/4-2-1.png" width="500" height="500" />
asyncio 不是多进程, 也不是多线程, 单单是一个线程, 但是是在 Python 的功能间切换着执行. 切换的点用 await 来标记, 能够异步的功能用 async 标记, 比如 async def function():<br>
写程序比较asyncio与顺序执行的不同

In [ ]:
#普通程序的顺序执行
import time
def job(t):
    print('start job:',t)
    time.sleep(t)
    print('job ',t,' takes ',t,'s.')

def main():
    [job(i) for i in range(1,3)] #执行1s和2s任务
    
start=time.time()
if __name__=='__main__':
    main()
print('total time is ',time.time()-start,'s')    

In [ ]:
import asyncio
async def job(t):
    print('start job ',t)
    await asyncio.sleep(t)
    print('job ',t,' takes ',t,'s')

async def main(loop):
    tasks=[loop.create_task(job(i)) for i in range(1,3)]
    await asyncio.wait(tasks)
    
start=time.time()
if __name__=='__main__':
    loop=asyncio.get_event_loop()
    loop.run_until_complete(main(loop))
print('using asyncio total time is ',time.time()-start,'s')   


### 与aiohttp合用<br>
上边的asyncio的性质可以说非常适合爬虫了.**aiohttp**是一个配合asyncio使用的工具,有爬取网站的功能.

In [24]:
#我抄的,没搞熟
import aiohttp
import asyncio
import time
async def job(session):
    response=await session.get(URL) #爬取session相应的网站
    return str(response.url)  #传回session网站

async def main(loop):
    async with aiohttp.ClientSession() as session:  #创建一个session,推荐使用session
        tasks=[loop.create_task(job(session)) for _ in range(2)] #创建任务
        finished,unfinished=await asyncio.wait(tasks)  #读取任务情况,并输出
        print([f.result() for f in finished])

start=time.time()
if __name__=='__main__':
    URL = 'https://morvanzhou.github.io/'
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main(loop))
print('open morvanpage 2times total time:',time.time()-start)        

['https://morvanzhou.github.io/', 'https://morvanzhou.github.io/']
open morvanpage 2times total time: 0.9378917217254639


### asyncio与multiprocessing合用<br>
<img src='https://morvanzhou.github.io/static/results/scraping/4-2-3.png' height=700 width=500>
两者的优势互补,结果会更强.